In [42]:
from transformers import pipeline
import pandas as pd 
import torch

In [46]:
import os 
data_dir = '/home/ubuntu/fin_experiment/data/competition_data'
data_combined_news = pd.read_csv(os.path.join(data_dir, 'data_combined_news.csv'), index_col=0)

In [47]:
data_combined_news.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,Top26
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge""","[""b'Georgian troops retreat from S. Osettain c..."
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo...","[""b'Russia angered by Israeli military sale to..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man...","[""b'U.S. Beats War Drum as Iran Dumps the Doll..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...,"['b""Bush announces Operation Get All Up In Rus..."
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal

In [48]:
data_combined_news.rename(columns={'Reddit': 'Top26'}, inplace=True)
data_combined_news.fillna('', inplace=True)

In [49]:
assert len(data_combined_news[data_combined_news.isna().any(axis=1)]) == 0

In [51]:
data_combined_news['Date'] = pd.to_datetime(data_combined_news['Date'])

In [52]:
data_combined_news.to_csv(os.path.join(data_dir, 'data_combined_news.csv'))

In [63]:
texts = []
keys = [f'Top{i}' for i in range(1, 26)]
for i, row in data_combined_news.filter(like='Top').iterrows():
    for key in keys:
        texts.append(row[key])
    texts.extend(eval(row['Top26']))


In [64]:
vectorizer = TfidfVectorizer(min_df=5, stop_words='english')
dtm = vectorizer.fit_transform(texts)

### Removing duplicates

Tested multiple ways of removing exact/almost duplicates. Eventually used filter with semantic similarity scoring

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def remove_near_duplicates_tfidf(row, similarity_threshold=0.8):
  """Removes near duplicate news items based on TF-IDF and cosine similarity."""

  texts = list(row[:-1].values) # exclude Reddit
  reddit_texts = eval(row[-1])
  texts = texts + reddit_texts
  tfidf_matrix = vectorizer.transform(texts)
  keep_indices = []
  for i in range(len(texts)):
    is_duplicate = False
    for j in keep_indices:
      similarity = cosine_similarity(tfidf_matrix[i], tfidf_matrix[j])[0][0]
      if similarity >= similarity_threshold:
        is_duplicate = True
        break
    if not is_duplicate:
      keep_indices.append(i)

  return [texts[i] for i in keep_indices]

unique_headlines = data_combined_news.filter(like='Top').apply(remove_near_duplicates_tfidf, axis=1)

In [11]:
# Setting device on GPU if available
TOKENIZERS_PARALLELISM = False
torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

torch.backends.cudnn.deterministic = True


Using device: cuda

Tesla T4
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [74]:
from sentence_transformers import SentenceTransformer
model_sent = SentenceTransformer('all-mpnet-base-v2')  # Or another suitable model
model_sent.to(device)

def remove_semantically_similar(row, similarity_threshold=0.7):
    """Removes semantically similar news items using Sentence-Transformers."""
    texts = list(row[:-1].values) # exclude Reddit
    reddit_texts = eval(row.iloc[-1])
    texts = texts + reddit_texts
    embeddings = model_sent.encode(texts)

    keep_indices = []
    for i in range(len(texts)):
        is_duplicate = False
        for j in keep_indices:
            similarity = cosine_similarity([embeddings[i]], [embeddings[j]])[0][0]
            if similarity >= similarity_threshold:
                is_duplicate = True
                break
        if not is_duplicate:
            keep_indices.append(i)
    return [texts[i] for i in keep_indices]


In [75]:
def remove_semantically_similar_optimized(texts, similarity_threshold=0.7):
    """Removes semantically similar texts using vectorized cosine similarity."""
    if not texts:
        return []

    embeddings = model_sent.encode(texts)
    unique_indices = []
    if embeddings.size > 0:
        unique_embeddings = [embeddings[0]]
        unique_indices.append(0)

        for i in range(1, len(embeddings)):
            similarity_scores = cosine_similarity([embeddings[i]], unique_embeddings)[0]
            if np.all(similarity_scores < similarity_threshold):
                unique_embeddings.append(embeddings[i])
                unique_indices.append(i)

    return [texts[i] for i in unique_indices]

def process_row_optimized(row, similarity_threshold=0.7):
    texts = list(row[:-1].values)  # exclude Reddit
    reddit_texts = eval(row.iloc[-1])
    all_texts = texts + reddit_texts
    return remove_semantically_similar_optimized(all_texts, similarity_threshold)

unique_headlines2 = data_combined_news.filter(like='Top').apply(process_row_optimized, axis=1)

In [115]:
data_combined_news['unique_headlines'] = pd.DataFrame(unique_headlines2.values, index=data_combined_news.index)

In [72]:
# Removing duplicates 
def remove_exact_duplicates(news_list):
  """Removes exact duplicate news items from a list."""
  return list(set(news_list))

news_headlines = ["Breaking News: Market Surges", "Market Surges", "Breaking News: Market Surges", "Another Story"]
unique_headlines = remove_exact_duplicates(news_headlines)
print(unique_headlines)

In [ ]:
from difflib import SequenceMatcher

def is_similar(s1, s2, threshold=0.9):
  """Checks if two strings are similar based on SequenceMatcher."""
  return SequenceMatcher(None, s1, s2).ratio() >= threshold

def remove_near_duplicates(news_list, similarity_threshold=0.9):
  """Removes near duplicate news items based on string similarity."""
  unique_news = []
  for i, news1 in enumerate(news_list):
    is_duplicate = False
    for news2 in unique_news:
      if is_similar(news1, news2, similarity_threshold):
        is_duplicate = True
        break
    if not is_duplicate:
      unique_news.append(news1)
  return unique_news

news_headlines = ["Company X Announces Record Profits", "Company X reports record profits.", "Market Reacts to Earnings", "Another Story"]
unique_headlines = remove_near_duplicates(news_headlines)
print(unique_headlines)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def remove_near_duplicates_tfidf(news_list, similarity_threshold=0.8):
  """Removes near duplicate news items based on TF-IDF and cosine similarity."""
  if not news_list:
    return []

  vectorizer = TfidfVectorizer()
  tfidf_matrix = vectorizer.fit_transform(news_list)

  keep_indices = []
  for i in range(len(news_list)):
    is_duplicate = False
    for j in keep_indices:
      similarity = cosine_similarity(tfidf_matrix[i], tfidf_matrix[j])[0][0]
      if similarity >= similarity_threshold:
        is_duplicate = True
        break
    if not is_duplicate:
      keep_indices.append(i)

  return [news_list[i] for i in keep_indices]

# Example usage:
news_headlines = ["Company Y's stock price soars after earnings beat.", "Stock price of Company Y jumps following better-than-expected earnings.", "Interest rates remain unchanged.", "Unrelated news."]
unique_headlines = remove_near_duplicates_tfidf(news_headlines)
print(unique_headlines)

### Use FinBert sentiment

In [132]:
# Load pre-trained FinBERT model and tokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import numpy as np

finbert_model_name = "ProsusAI/finbert"
finbert_tokenizer = AutoTokenizer.from_pretrained(finbert_model_name)
finbert_model = AutoModelForSequenceClassification.from_pretrained(finbert_model_name)
finbert_model.to(device)

def get_finbert_sentiment_scores(row):
    """
    Args:
        headlines (list of str): A list of news headlines.

    Returns:
        dict: A dictionary containing the average FinBERT sentiment scores
              ('positive', 'negative', 'neutral').
    """
    inputs = finbert_tokenizer(row, return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = finbert_model(**inputs)
    scores = outputs.logits.detach().cpu().numpy()
    averaged_per_day = np.sum(softmax(scores, axis=1), axis=0) / len(scores)
    return {
        "finbert_positive": averaged_per_day[0],
        "finbert_negative": averaged_per_day[1],
        "finbert_neutral": averaged_per_day[2],
    }

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [133]:
# Apply FinBERT sentiment analysis to news headlines
news_finbert_sentiment = data_combined_news['unique_headlines'].apply(get_finbert_sentiment_scores)

In [134]:
news_finbert_sentiment.head()

0    {'finbert_positive': 0.08257684, 'finbert_nega...
1    {'finbert_positive': 0.19303668, 'finbert_nega...
2    {'finbert_positive': 0.07998301, 'finbert_nega...
3    {'finbert_positive': 0.09594671, 'finbert_nega...
4    {'finbert_positive': 0.12295767, 'finbert_nega...
Name: unique_headlines, dtype: object

In [135]:
data_combined_news[['finbert_positive', 'finbert_negative', 'finbert_neutral']] = pd.DataFrame(news_finbert_sentiment.tolist(), index=data_combined_news.index)

In [136]:
data_combined_news.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top21,Top22,Top23,Top24,Top25,Top26,unique_headlines,finbert_positive,finbert_negative,finbert_neutral
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge""","[""b'Georgian troops retreat from S. Osettain c...","[b""Georgia 'downs two Russian warplanes' as co...",0.082577,0.321242,0.596181
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo...","[""b'Russia angered by Israeli military sale to...",[b'Why wont America and Nato help us? If they ...,0.193037,0.264102,0.542861
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man...","[""b'U.S. Beats War Drum as Iran Dumps the Doll...",[b'Remember that adorable 9-year-old who sang ...,0.079983,0.206084,0.713933
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...,"['b""Bush announces Operation Get All Up In Rus...",[b' U.S. refuses Israel weapons to attack Iran...,0.095947,0.310377,0.593676
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree to missle defense deal. ...,"b'Will the Russians conquer Tblisi? Bet on it,...",...,b'Saudi Arabia: Mother moves to block child ma...,b'Taliban wages war on humanitarian aid workers',"b'Russia: World ""can forget about"" Georgia\'s...",b'Darfur rebels accuse Sudan of mounting major...,b'Philippines : Peace Advocate say Muslims nee...,"[""b'Poland and US agree to missle def

### Use Vader sentiment

In [137]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def get_headline_sentiment(row):
    compound_scores = [analyzer.polarity_scores(t)['compound'] for t in row]
    return np.mean(compound_scores)

news_vader_sentiment = data_combined_news['unique_headlines'].apply(get_headline_sentiment)

In [138]:
data_combined_news['vader_sentiment_compound'] = pd.DataFrame(news_vader_sentiment.tolist(), index=data_combined_news.index)


### Get topics (?)

In [26]:
lda = LatentDirichletAllocation(n_components=10, random_state=42)
lda.fit(dtm)


LatentDirichletAllocation(random_state=42)

In [140]:
# Extract top words for each topic
num_words = 10
vocab = vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    print(f"Topic {topic_idx}:")
    top_words = topic.argsort()[-num_words:][::-1]  # Get indices of top words
    for i in top_words:
        print(f"  {vocab[i]}: {topic[i]}")

Topic 0:
  oil: 162.270203391444
  china: 127.06753833529046
  plane: 115.5185496559688
  russian: 100.1759214518697
  new: 96.79327081414287
  chinese: 90.95948219479375
  says: 89.31326069678911
  climate: 86.29719795580498
  russia: 83.39987994957146
  world: 83.24544337908023
Topic 1:
  pirate: 141.66649569349377
  snowden: 126.79117017949818
  world: 98.55785381839591
  eu: 91.67982444617581
  russia: 85.88377143351642
  bay: 85.00783195458371
  says: 83.38234793497907
  edward: 77.61203822984956
  new: 75.68791428541019
  party: 73.30473111604618
Topic 2:
  sex: 141.75695767308568
  pope: 131.31521566695233
  marriage: 113.07165422869822
  minister: 112.41226031315466
  gay: 109.54689901052056
  church: 102.62222960316592
  abuse: 101.07997187750132
  prime: 100.11398422005138
  catholic: 99.49123304605503
  david: 92.40332900695685
Topic 3:
  police: 168.25543186178004
  government: 130.94170266493776
  rights: 128.0070957940006
  protesters: 120.235198224297
  israel: 115.30490

In [141]:
def get_headline_topics(row):
    m = vectorizer.transform(row)
    topic_distribution = lda.transform(m)
    return topic_distribution.mean(axis=0)  # Average topic distribution

In [142]:
headline_topics = data_combined_news['unique_headlines'].apply(get_headline_topics)

In [143]:
data_combined_news[[f'headline_topic_{i}' for i in range(10)]] = pd.DataFrame(headline_topics.tolist(), index=data_combined_news.index)

## Add market data

In [145]:
data_djia = pd.read_csv(os.path.join(data_dir, 'DJIA_table(train).csv'))
data_djia['Date'] = pd.to_datetime(data_djia['Date'])

/tmp/ipykernel_2579/4018146104.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data_djia['Date'] = pd.to_datetime(data_djia['Date'])


In [147]:
data_djia.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2015-12-31,17590.66016,17590.66016,17421.16016,17425.02930,93690000,17425.02930
1,2015-12-30,17711.93945,17714.13086,17588.86914,17603.86914,59760000,17603.86914
2,2015-12-29,17547.36914,17750.01953,17547.36914,17720.98047,69860000,17720.98047
3,2015-12-28,17535.66016,17536.90039,17437.33984,17528.26953,59770000,17528.26953
4,2015-12-24,17593.25977,17606.33984,17543.94922,17552.16992,40350000,17552.16992


In [148]:
merged_data = pd.merge(data_djia, data_combined_news, on='Date', how='left')
merged_data = merged_data.rename(columns={'Unnamed: 0': 'text_original_index'})

In [149]:
merged_data = merged_data.sort_values(by='Date')

In [150]:
merged_data.head()

,Date,Open,High,Low,Close,Volume,Adj Close,Label,Top1,Top2,...,headline_topic_0,headline_topic_1,headline_topic_2,headline_topic_3,headline_topic_4,headline_topic_5,headline_topic_6,headline_topic_7,headline_topic_8,headline_topic_9
1862,2008-08-08,11432.08984,11759.95996,11388.04004,11734.32031,212830000,11734.32031,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',...,0.077452,0.027559,0.089896,0.054157,0.137296,0.137123,0.227256,0.055558,0.134330,0.059373
1861,2008-08-11,11729.66992,11867.11035,11675.53027,11782.34961,183190000,11782.34961,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',...,0.112519,0.106180,0.089383,0.037320,0.143491,0.097843,0.282645,0.042399,0.030168,0.058052
1860,2008-08-12,11781.70020,11782.34961,11601.51953,11642.46973,173590000,11642.46973,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""",...,0.101699,0.028118,0.036736,0.081626,0.028117,0.111068,0.311158,0.107205,0.108571,0.085703
1859,2008-08-13,11632.80957,11633.78027,11453.33984,11532.95996,182550000,11532.95996,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",...,0.066877,0.127792,0.065496,0.073997,0.123949,0.070983,0.184402,0.040296,0.134380,0.111827
1858,2008-08-14,11532.07031,11718.28027,11450.88965,11615.92969,159790000,11615.92969,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,...,0.067761,0.053651,0.108737,0.069179,0.238253,0.028422,0.124376,0.117796,0.163402,0.028423


In [152]:
merged_data.to_csv(os.path.join(data_dir, 'full_data.csv'))